In [1]:
from utils import *
import os
import numpy as np

Setup_environment()

from hpe_wrapper import Wrapper_2Dpose, Wrapper_3Dpose


# Define the path for the data_3d file
data_3d_path = 'data/intermedia/data_3d.npz'

# Check if the data_3d file exists
if os.path.exists(data_3d_path):
    # Load the data_3d if it exists
    loaded_data = np.load(data_3d_path)
    data_3d = loaded_data['data_3d']
else:
    # Your existing code to generate data_3d
    model_2D ='./detectron/configs/COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml'
    weights_2D = './detectron/checkpoint/model_final_997cc7.pkl'
    model_3D = './VideoPose3D/checkpoint/pretrained_h36m_detectron_coco.bin'

    pose2d = Wrapper_2Dpose(model=model_2D, weights=weights_2D, ROI_thr=0.75)
    pose_3d = Wrapper_3Dpose(model_3D)

    config = load_config()
    print(config['video_object'])

    video_object = Video_wrapper(config['video_object'], resize_video_by=0.3)
    data_2d, metadata_vid = pose2d.predict_2D_poses(input_video_object=video_object)
    data_3d = pose_3d.predict_3D_poses(data_2d, metadata_vid)

    # Save the data_3d to an .npz file for future use
    np.savez(data_3d_path, data_3d=data_3d)

## Pose Optim  Loop

In [2]:
import torch
from tensor_ergonomics import RULAXXX

def run_pose_optimization(original_pose, num_steps=3):
    """
    Optimize the pose using the RULA score.
    
    Parameters:
    - original_pose: A tensor of shape (17, 3)
    
    Returns:
    - Optimized pose.
    """
    #torch.autograd.set_detect_anomaly(True)
    if not isinstance(original_pose, torch.Tensor):
        original_pose = torch.tensor(original_pose)

    original_pose = original_pose.clone().detach().requires_grad_(True)

    optimizer = torch.optim.Adam([original_pose], lr=0.1) #(1,200t, 17, 3)

    for step in range(num_steps):
        optimizer.zero_grad()

        rula_eval = RULAXXX(original_pose)
        rula_score = rula_eval.compute_scores()
        print(rula_score.grad_fn)
        ergo_sum = rula_score.sum()
        loss = ergo_sum
        
        loss.backward()
        print(original_pose.grad)
        optimizer.step()
        print(f"Step {step}: Loss: {loss.item()}, RULA Score: {rula_score.sum().item()}")
        
    return original_pose

optimized_pose = run_pose_optimization(data_3d)

p = optimized_pose.detach().numpy()
print('Original pose: ', data_3d[1,1,])
print('Optim pose: ', p[1,1,:])

Angles: torch.Size([168, 14])
--- Frame 0: ---
Frame: tensor([ 14.3200, 111.9615,  36.7538, 119.7843,  29.0617,  43.0275,   9.8064,
         23.7113,  14.8478,   1.2035,  91.7161,  35.1584,  99.9522,  98.7746],
       grad_fn=<UnbindBackward0>)
Max shoulder: tensor(36.7538, grad_fn=<MaximumBackward0>)
Max abduction: tensor(119.7843, grad_fn=<MaximumBackward0>)
Max elbow: tensor(43.0275, grad_fn=<MaximumBackward0>)
Score Upper Arm: tensor([[2.]], grad_fn=<AddBackward0>)
Score Lower Arm: tensor([1.], grad_fn=<CatBackward0>)
Current Score A: tensor([2.])
Score Neck: tensor([1.], grad_fn=<CatBackward0>)
Score Trunk: tensor([1.], grad_fn=<CatBackward0>)
Min knee: tensor(9.8064, grad_fn=<MinimumBackward0>)
Score Legs: tensor([1.], grad_fn=<CatBackward0>)
Current Score B: tensor(1.)
Clamped Curr A B score for index:  tensor([2.], requires_grad=True) tensor(1., requires_grad=True)
Index 1 and 2:  tensor([1]) tensor(0)
Current Total Score: tensor([2.])
Total Score: [tensor([2.])]
--- Frame 1: -

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [3]:
#FAKE
import numpy as np
from utils import *
import os
import numpy as np
Setup_environment()

from hpe_wrapper import Wrapper_2Dpose, Wrapper_3Dpose


# Define the path for the data_3d file
data_3d_path = 'data/intermedia/data_3d.npz'

# Check if the data_3d file exists
if os.path.exists(data_3d_path):
    # Load the data_3d if it exists
    loaded_data = np.load(data_3d_path)
    data_3d = loaded_data['data_3d']
else:
    # Your existing code to generate data_3d
    model_2D ='./detectron/configs/COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml'
    weights_2D = './detectron/checkpoint/model_final_997cc7.pkl'
    model_3D = './VideoPose3D/checkpoint/pretrained_h36m_detectron_coco.bin'

    pose2d = Wrapper_2Dpose(model=model_2D, weights=weights_2D, ROI_thr=0.75)
    pose_3d = Wrapper_3Dpose(model_3D)

    config = load_config()
    print(config['video_object'])

    video_object = Video_wrapper(config['video_object'], resize_video_by=0.3)
    data_2d, metadata_vid = pose2d.predict_2D_poses(input_video_object=video_object)
    data_3d = pose_3d.predict_3D_poses(data_2d, metadata_vid)

    # Save the data_3d to an .npz file for future use
    np.savez(data_3d_path, data_3d=data_3d)
    
from FAKE import FAKE

def run_FAKE_optimization(fake_pose, num_steps=3):

    #torch.autograd.set_detect_anomaly(True)
    if not isinstance(fake_pose, torch.Tensor):
        fake_pose = torch.tensor(fake_pose)

    fake_pose = fake_pose.clone().detach().requires_grad_(True)

    optimizer = torch.optim.Adam([fake_pose], lr=0.1) #(1,200t, 17, 3)

    for step in range(num_steps):
        optimizer.zero_grad()

        fake_eval = FAKE(fake_pose)
        fake_score = fake_eval.compute_scores()
        print('1 fake_score.grad_fn: ', fake_score.grad_fn)
        ergo_sum = fake_score.sum()
        loss = ergo_sum

        loss.backward()
        print(fake_pose.grad)
        optimizer.step()
        print(f"Step {step}: Loss: {loss.item()}, FAKE Score: {fake_score.sum().item()}")

    return fake_pose

optimized_pose = run_FAKE_optimization(data_3d)

p = optimized_pose.detach().numpy()
print('Original pose: ', data_3d[1,1,])
print('Optim pose: ', p[1,1,:])

--- Frame 0: ---
Frame: tensor([ 14.3200, 111.9615,  36.7538, 119.7843,  29.0617,  43.0275,   9.8064,
         23.7113,  14.8478,   1.2035,  91.7161,  35.1584,  99.9522,  98.7746],
       grad_fn=<UnbindBackward0>)
Frame requires_grad: True
Score Upper Arm requires_grad: True
Score Lower Arm requires_grad: True
Max shoulder: tensor(36.7538, grad_fn=<MaximumBackward0>)
Sscore_valrequires_grad: True
Max abduction: tensor(119.7843, grad_fn=<MaximumBackward0>)
Max elbow: tensor(43.0275, grad_fn=<MaximumBackward0>)
Score Upper Arm: tensor([[2.]], grad_fn=<AddBackward0>)
Score Lower Arm: tensor([1.], grad_fn=<CatBackward0>)
Current Score A: tensor([2.], grad_fn=<IndexBackward0>)
Score Neck: tensor([1.], grad_fn=<CatBackward0>)
Score Trunk: tensor([1.], grad_fn=<CatBackward0>)
Min knee: tensor(9.8064, grad_fn=<MinimumBackward0>)
Score Legs: tensor([1.], grad_fn=<CatBackward0>)
Current Score B: tensor(1., grad_fn=<SelectBackward0>)
Clamped Curr A B score for index:  tensor([2.], grad_fn=<Clamp

In [10]:
data_3d.shape

(168, 17, 3)

In [ ]:
import plotly.graph_objects as go
import webbrowser
import time

def visualize_frame(frame):
    # Extract x, y, z coordinates of keypoints
    x = frame[:, 0]
    y = frame[:, 2]
    z = - frame[:, 1]

    # Create a scatter plot for the keypoints
    scatter = go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='markers',
        marker=dict(size=5, color='red')
    )

    # Define pairs of keypoints to connect with green lines
    pairs = [(0,1), (0,4), (0,7), (7,8), (8,9), (9,10), (4,5), (1,2), (5,6), (2,3),
             (8,11), (8,14), (11,12), (14,15), (12,13), (15,16)]

    lines = []
    for pair in pairs:
        lines.append(
            go.Scatter3d(
                x=[x[pair[0]], x[pair[1]]],
                y=[y[pair[0]], y[pair[1]]],
                z=[z[pair[0]], z[pair[1]]],
                mode='lines',
                line=dict(color='green', width=5)
            )
        )

    # Create the layout and add all traces
    layout = go.Layout(
        scene=dict(
            xaxis=dict(nticks=10, range=[min(x), max(x)]),
            yaxis=dict(nticks=10, range=[min(y), max(y)]),
            zaxis=dict(nticks=10, range=[min(z), max(z)])
        )
    )

    fig = go.Figure(data=[scatter] + lines, layout=layout)

    # Save the plot as an HTML file
    filename = "plot.html"
    fig.write_html(filename)

    # Open the saved HTML file in the default web browser
    webbrowser.open(filename)
    time.sleep(3)

# Example usage
# Assuming data_3d is your 3D array of shape (199, 17, 3)
for frame in p[9:10,:,:]:
    visualize_frame(frame)

In [ ]:
import torch
from tensor_ergonomics import RULAXXX

def run_pose_optimization(original_pose, num_steps=300):
    """
    Optimize the pose using the RULA score.
    
    Parameters:
    - original_pose: A tensor of shape (17, 3)
    
    Returns:
    - Optimized pose.
    """

    # Initialize the pose to be optimized
    pose = original_pose.clone().detach().requires_grad_(True)

    # Define an optimizer. Here, we use the Adam optimizer.
    optimizer = torch.optim.Adam([pose], lr=0.01)

    for step in range(num_steps):
        optimizer.zero_grad()

        # Compute the RULA score for the current pose            
        rula_eval = RULAXXX(original_pose)
        print(rula_eval)
        rula_score = rula_eval.compute_scores()
        
        # We want to minimize the RULA score, so we negate it for optimization
        loss = -rula_score

        # Backpropagate the loss
        loss.backward()

        # Update the pose
        optimizer.step()

        # Print the RULA score every 50 steps
        if step % 50 == 0:
            print(f"Step {step}: RULA Score: {rula_score.item()}")

    return pose


# Optimize the pose
original_pose = data_3d
original_pose_tensor = torch.tensor(original_pose, dtype=torch.float32, requires_grad=True)
print(original_pose_tensor.shape) #torch.Size([17, 3])
optimized_pose = run_pose_optimization(original_pose_tensor)
